In [1]:
import os

os.chdir("/home/tim/Development/OCPPM/")
import pprint

import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from loan_application_experiment.feature_encodings.baselines import metrics_helpers

# ONLY OBJECT TYPE==APPLICATION WILL BE CONSIDERED
oft_out_file = "data/BPI17/feature_encodings/OFT/application_features.csv"

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.3.1



In [2]:
oft = pd.read_csv(oft_out_file)
scaler = StandardScaler()
oft_values = scaler.fit_transform(oft.values)
oft = pd.DataFrame(oft_values, columns=oft.columns)
oft

,object_attr_value_event_LoanGoal_Debtrestructuring,object_attr_value_event_LoanGoal_Homeimprovement,object_attr_value_event_LoanGoal_Unknown,object_attr_value_event_LoanGoal_Existingloantakeover,object_attr_value_event_LoanGoal_Remainingdebthome,object_attr_value_event_LoanGoal_Notspeficied,object_attr_value_event_LoanGoal_Taxpayments,object_attr_value_event_LoanGoal_Otherseeexplanation,object_attr_value_event_LoanGoal_Extraspendinglimit,object_attr_value_event_LoanGoal_Car,object_attr_value_event_LoanGoal_Motorcycle,object_attr_value_event_LoanGoal_Businessgoal,object_attr_value_event_LoanGoal_Boat,object_attr_value_event_LoanGoal_CaravanCamper,object_attr_value_event_ApplicationType_Limitraise,object_attr_value_event_ApplicationType_Newcredit,event_num_event_RequestedAmount,object_lifecycle_duration
0,-0.007967,-0.567174,-0.284866,2.150721,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,-0.648491,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,0.244213,-0.663785
1,-0.007967,1.763128,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,-0.648491,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,-0.404211,-1.212790
2,-0.007967,1.763128,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,-0.648491,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,-0.079999,-0.696869
3,-0.007967,-0.567174,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,1.542042,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,-0.728423,0.396572
4,-0.007967,1.763128,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,-0.648491,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,1.216850,0.764011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31504,-0.007967,-0.567174,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,1.542042,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,-0.728423,0.061321
31505,-0.007967,-0.567174,-0.284866,2.150721,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,-0.648491,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,-0.079999,-1.533732
31506,-0.007967,1.763128,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,-0.648491,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,-0.404211,0.059856
31507,-0.007967,-0.567174,-0.284866,-0.464960,-0.165699,-0.187035,-0.069623,-0.323495,-0.142257,1.542042,-0.093832,-0.030871,-0.080125,-0.108856,-0.347159,0.347159,0.892637,0.083059


In [3]:
# make train test split
X, y = (
    oft.drop("object_lifecycle_duration", axis=1),
    oft.loc[:, "object_lifecycle_duration"],
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=0
)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

In [4]:
params = {
    "objective": "regression",
    "metric": ["mse", "mae", "mape", "rmse"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
}

bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)

[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 290
[LightGBM] [Info] Number of data points in the train set: 25207, number of used features: 16
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Info] Start training from score 0.000478


/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[25]	valid_0's l2: 0.978653	valid_0's l1: 0.803528	valid_0's mape: 0.688219	valid_0's rmse: 0.989269


In [5]:
y_train_preds = bst.predict(X_train)
y_valid_preds = bst.predict(X_valid)

eval_results = metrics_helpers.get_regression_evaluation(
    y_train, y_train_preds, y_valid, y_valid_preds
) | {"params": params}

pprint.pprint(eval_results)

{'params': {'metric': ['mse', 'mae', 'mape', 'rmse'],
            'num_boost_round': 100,
            'objective': 'regression',
            'stopping_rounds': 100},
 'train': {'MAE': 0.7959676618937744,
           'MAPE': 1.6078683188056646,
           'MSE': 0.9496621830284584,
           'R^2': 0.04741747161709642},
 'valid': {'MAE': 0.8035280541113625,
           'MAPE': 1.4143303969509002,
           'MSE': 0.9786530352377317,
           'R^2': 0.03319783435235868}}
